# AI ASSISTANTS 
This project has been created for the cohere hackathon hosted by lablab.ai and aims to create an AI Assistant. For this project the AI Assistant acts as customer support for an electronics company

In [ ]:
!pip install cohere

In [1]:
# All import statements
import cohere
import config
import examples_t
from examples_t import classify1
from cohere.classify import Example
import numpy as np
import config.py
from config.py import apikey
import streamlit as st 
import time
co = cohere.Client('apikey')

st.title("ODR AI ASSISTANTS")

In [10]:
### DATA ###
warranty_data=[Example("No I dont have warranty", "N"), Example("I used to but not anymore", "N"), 
               Example("It expired last month", "N"), Example("I will in a month", "N"), 
               Example("Yes I do", "Y"), Example("Just got it last month", "Y"), Example("I do", "Y"), 
               Example("I was planning on getting it next month", "N")]

#IMPORTANT VARIABLES
final_cat = "" #Category chosen by user
final_warr = "" #If user has warranty
final_feel = "" #How the user felt

In [5]:
# Here we get the response classification
first_input = st.text_input("Enter your Query: ")
input = first_input
tru_input = [input]
response = co.classify(
    model = "large",
    inputs = tru_input,
    examples= classify1
)
list = [] #This list will be used to summarise the whole interaction

tru_response = response.classifications[0].prediction #Getting the prediction with the highest confidence
list.append(tru_response) 

In [7]:
print(list)

['0']


In [11]:
a = tru_input
tec_gen = ""
def gen():
    global tec_gen
    tec_gen = co.generate(
        model = "command-xlarge-nightly",
        prompt = a[0],        
        max_tokens=300,
        stop_sequences= ['--'],
        temperature=0.9,
        num_generations= 1)
    

In [10]:
if list[0] == '0' or list[0] == '1':
    st.write("Process running...")
    time.sleep(30)
    global tec_gen
    tec_gen = co.generate(
        model = "command-xlarge-nightly",
        prompt = a[0],        
        max_tokens=300,
        stop_sequences= ['--'],
        temperature=0.9,
        num_generations= 1)
    st.write(print(tec_gen.generations[0].text))
elif list[0] == '4' or list[0] == '3':
    print("Thankyou for connecting, based on your query we will now connect you to an agent who will further assist you")
elif list[0] == '5':
    warranty_input = input("Do you have warranty?")
    warranty = co.classify(
    model='large',
    inputs= warranty_input,
    examples= warranty_data
    )
    if warranty == 0: # 0 means the user doesnt have any warranty
        print("That's okay. You can visit our stores to get your device repaired, I will connect you to an agent to calculate the cost")
        final_warr = "The user does not have warranty"
    else:
        print("Alright, you can come to our store or get a repairman to visit you and get your device repaired free of cost")
        final_warr = "The user has warranty"
elif list[0] == '6':
    Rinput = st.text_input("Enter some text")
    response = co.classify(
    model='large',
    inputs= [Rinput],
    examples=[Example("Great product, I loved it", "8"), Example("Thankyou for being so awesome", "8"), Example("Absolute trash", "9"), Example("This really sucked", "9"), Example("I hope your company burns", "9"),]
    )
    feelings = response.classifications[0].prediction
    if feelings == 8:
        st.write("Thankyou for your valuable feedback! We will ensure you continue to recieve quality products from us!")
        final_feel = "The user is happy with the company"
    else:
        st.write("We are sorry to hear that, please contact us at greivances@lenuwu.com with more details")
        final_feel = "The user is unhappy with the company"

In [9]:
#implementation of text map   
category = ""
response_map = {'0': "Technical support", '1': "Product information", '2': "Order Inquiries", '3': "Returns and Exchanges", '4': "Billing and account issues", '5': "Warranty and repair", '6':"Complaints and feedback"}
print(list)
for i in list:
    category = response_map.get(i, "")
    final_cat = "The user contacted the " + category + " department"


['0']
The user contacted the Technical support department


# ANALYSIS
Here we see the summary of the interaction

In [ ]:
print(final_cat + final_warr + final_feel)
